In [62]:
import os
import sys

import tensorflow as tf
import tensorflow.keras as keras

import numpy as np
import csv
import pandas as pd


In [52]:
with open ("../data/ml-100k/u1.base", "r") as f:
    data_train = list(csv.reader(f, delimiter="\t"))
with open ("../data/ml-100k/u1.test", "r") as f:
    data_test = list(csv.reader(f, delimiter="\t"))
data_train = np.array(data_train)
data_test = np.array(data_test)

film_dim_test = np.amax(np.array(data_test[:,1]).astype(np.float)) + 1
user_dim_test = np.amax(np.array(data_test[:,0]).astype(np.float)) + 1
print("film size test ", film_dim_test)
print("user size size test", user_dim_test)
mat_test = np.zeros((int(user_dim_test), int(film_dim_test)))

film_dim_train = np.amax(np.array(data_train[:,1]).astype(np.float)) + 1
user_dim_train = np.amax(np.array(data_train[:,0]).astype(np.float)) + 1
print("film size train ", film_dim_train)
print("user size size train", user_dim_train)
mat_train = np.zeros((int(user_dim_train), int(film_dim_train)))

film size test  1592.0
user size size test 463.0
film size train  1683.0
user size size train 944.0


In [55]:
for d in data_train:
    user = int(d[0])
    movie = int(d[1])
    rating = int(d[2])
    mat_train[user,movie] = rating
df_train = pd.DataFrame(mat_train)

for d in data_test:
    user = int(d[0])
    movie = int(d[1])
    rating = int(d[2])
    mat_test[user,movie] = rating
df_test = pd.DataFrame(mat_test)

In [56]:
print(df_train.shape)
print(df_test.shape)

(944, 1683)
(463, 1592)


In [60]:
train_x = []
train_y = []
test_x = []
test_y = []
max_length = 0
for user_id, ratings in df_train.iterrows():
    user = []
    user_ratings = []
    for movie_id, rating in enumerate(ratings):
        if rating > 0.0:
            user.append(movie_id + 1)
            user_ratings.append(rating)
    if max_length < len(user):
        max_length = len(user)
    train_x.append(user)
    train_y.append(user_ratings)
    
    
for user_id, ratings in df_test.iterrows():
    user = []
    user_ratings = []
    for movie_id, rating in enumerate(ratings):
        if rating > 0.0:
            user.append(movie_id + 1)
            user_ratings.append(rating)
    if max_length < len(user):
        max_length = len(user)
    test_x.append(user)
    test_y.append(user_ratings)

train_x =  tf.keras.preprocessing.sequence.pad_sequences(train_x,padding='post',maxlen=max_length)
train_y =  tf.keras.preprocessing.sequence.pad_sequences(train_y,padding='post',maxlen=max_length)
test_x =  tf.keras.preprocessing.sequence.pad_sequences(test_x,padding='post',maxlen=max_length)
test_y =  tf.keras.preprocessing.sequence.pad_sequences(test_y,padding='post',maxlen=max_length)

print(train_x.shape)
print(test_x.shape)


(944, 685)
(463, 685)


In [73]:
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional
from tensorflow.keras.initializers import Constant

class POSTagger(keras.Model):

    def __init__(self):
        super(POSTagger, self).__init__()
        # FIXME
        lstm_units = 64
        num_classes = 6
        self.emb = Embedding(
            input_dim=1683 + 1,
            output_dim=300,
            #embeddings_initializer=Constant(pretrained),
            mask_zero=True,
            trainable=True)

        self.lstm = Bidirectional(LSTM(
            units=lstm_units, return_sequences=True))

        self.dense = Dense(
            units=num_classes,
            activation='softmax')

    def call(self, x):
        # FIXME
        mask = self.emb.compute_mask(x)
        x = self.emb(x)
        x = self.lstm(x, mask=mask)
        x = self.dense(x)
        return x


model = POSTagger()
print("init done")
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])
print("compile done")
model.fit(
    x=train_x,
    y=train_y,
    batch_size=10,
    epochs=30,
    validation_data=(test_x, test_y))

init done
compile done
Train on 700 samples, validate on 244 samples
Epoch 1/30
700/700 [==============================] - 52s 75ms/sample - loss: 0.2242 - accuracy: 0.3409 - val_loss: 0.1850 - val_accuracy: 0.3679
Epoch 2/30
700/700 [==============================] - 42s 60ms/sample - loss: 0.2062 - accuracy: 0.3727 - val_loss: 0.1814 - val_accuracy: 0.3763
Epoch 3/30
700/700 [==============================] - 42s 60ms/sample - loss: 0.2017 - accuracy: 0.3905 - val_loss: 0.1807 - val_accuracy: 0.3776
Epoch 4/30
700/700 [==============================] - 42s 60ms/sample - loss: 0.1989 - accuracy: 0.4025 - val_loss: 0.1800 - val_accuracy: 0.3730
Epoch 5/30
340/700 [=============>................] - ETA: 20s - loss: 0.1923 - accuracy: 0.4112

KeyboardInterrupt: 

In [68]:
with open ("../data/ml-100k/u.data", "r") as f:
    data = list(csv.reader(f, delimiter="\t"))
data = np.array(data)

film_dim = np.amax(np.array(data[:,1]).astype(np.float)) + 1
user_dim = np.amax(np.array(data[:,0]).astype(np.float)) + 1
print("film size", film_dim)
print("user size size", user_dim)
mat = np.zeros((int(user_dim), int(film_dim)))

for d in data:
    user = int(d[0])
    movie = int(d[1])
    rating = int(d[2])
    mat[user,movie] = rating
df = pd.DataFrame(mat)

data_x = []
data_y = []

max_length = 0
for user_id, ratings in df.iterrows():
    user = []
    user_ratings = []
    for movie_id, rating in enumerate(ratings):
        if rating > 0.0:
            user.append(movie_id + 1)
            user_ratings.append(rating)
    if max_length < len(user):
        max_length = len(user)
    data_x.append(user)
    data_y.append(user_ratings)

data_x =  tf.keras.preprocessing.sequence.pad_sequences(data_x,padding='post',maxlen=max_length)
data_y =  tf.keras.preprocessing.sequence.pad_sequences(data_y,padding='post',maxlen=max_length)

print(data_x.shape)
print(data_y.shape)

train_x = data_x[:700]
test_x = data_x[700:]
train_y = data_y[:700]
test_y = data_y[700:]

print()
print(test_y.shape)
print(test_x.shape)
print(train_y.shape)
print(train_x.shape)


film size 1683.0
user size size 944.0
(944, 737)
(944, 737)

(244, 737)
(244, 737)
(700, 737)
(700, 737)
